In [34]:
import os
from credentials import get_credentials_gemini
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import textwrap
from pprint import pprint

In [2]:
def to_markdown(text):
    text = text.replace('•', ' *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
API_KEY = get_credentials_gemini()

In [5]:
genai.configure(api_key=get_credentials_gemini())

In [12]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                              system_instruction='you are a regular natural language user')

In [13]:
def get_completion(prompt, temperature=1.0):
    completion = model.generate_content(prompt,
            generation_config=genai.types.GenerationConfig(
            temperature=temperature)
    )
    return completion.text

In [19]:
#Creat a path to access the data
DATAPATH = os.path.join('..', 'data')

In [36]:
#Read the statute
with open(os.path.join(DATAPATH, '18USC924c1.txt'), 'r') as f:
    statute = f.read()

#Case summary from the majority opinion in Smith v. United States
with open(os.path.join(DATAPATH, 'smith_case_summary.txt'), 'r') as f:
    case_summary = f.read()

In [37]:
prompt = f''''
Read the following statute: {statute}.
Now, read the following case summary: {case_summary}.
Based on this case, would you say that a reasonable person would consider that Mr. Smith _used_ 
his gun in the sense that the language of the statute requires? 
Answer in the format:
ANSWER: Your answer (yes or no) 
REASONING: Your reasoning.
Use markdown to format your answer.
'''

In [23]:
#Completion with prompt and summary from SCOTUS
first_try = get_completion(prompt)

In [24]:
to_markdown(first_try)

> ## ANSWER: Yes 
> 
> ## REASONING: 
> 
> While Mr. Smith didn't fire or brandish the MAC-10 in a traditionally aggressive manner, his actions demonstrate that he **used** the firearm as leverage and a bartering tool during a drug trafficking crime. Let's break it down:
> 
> * **Context of Drug Trafficking:**  The entire scenario unfolded within the context of a drug deal. Mr. Smith traveled interstate with the intention to purchase and resell cocaine, a clear act of drug trafficking.
> * **Active Involvement of the Firearm:**  He actively brought the MAC-10 to the motel room and readily presented it to the undercover officer, showcasing its features and firepower. This wasn't merely possessing a firearm; it was actively incorporating it into the negotiation. 
> * **Bartering Tool:** He explicitly offered to trade the MAC-10 for two ounces of cocaine, demonstrating its use as a valuable asset within the drug deal. The firearm wasn't just present; it was a central element of the proposed transaction.
> * **Legislative Intent:** The statute aims to deter the presence and use of firearms in violent crimes and drug trafficking.  Mr. Smith's actions directly fall within this scope, as he actively utilized the firearm to facilitate and potentially advance his drug trafficking activities. 


In [25]:
#Case summary from Gries and Slocum paper
case_summary = '''
John Angus Smith offered to trade an automatic weapon to an undercover officer for cocaine.
'''

In [27]:
#Completion with prompt and summary from Gries and Slocum paper
secont_try = get_completion(prompt)

In [28]:
to_markdown(secont_try)

> ## ANSWER: No
> 
> ## REASONING:
> 
> While John Angus Smith certainly possessed and intended to transfer an automatic weapon (which falls under the statute's definition of a firearm), the act of offering it in a trade does not constitute "use" as it is typically understood within the context of the statute. The law generally interprets "use" of a firearm in the context of violence or drug trafficking crimes to involve its active employment to facilitate the crime, such as brandishing, firing, or otherwise leveraging it as a tool or threat during the criminal act. 
> 
> In this case, the firearm was the object of the transaction itself rather than a tool used to commit the drug trafficking crime (attempting to acquire cocaine). The trade offer could be seen as a separate act from the drug offense. 


In [29]:
#Varying the temperature
answers = dict()
for temp in [0.5, 1, 1.5]:
    answers[f'completion_temp_{temp}'] = get_completion(prompt, temperature=temp)

In [35]:
for key, value in answers.items():
    print(f'Here is the completion with temperature {key}:')
    pprint(value)
    print('\n##############################################\n')

Here is the completion with temperature completion_temp_0.5:
('## ANSWER: No\n'
 '\n'
 '## REASONING: \n'
 '\n'
 'While Mr. Smith clearly possessed and intended to transfer the automatic '
 'weapon, the scenario described does not indicate that he actively "used" the '
 'firearm in the context of violence or drug trafficking as defined by the '
 'statute. The statute focuses on the active employment of the firearm during '
 'the commission of a crime, such as brandishing, firing, or using it as a '
 'tool of intimidation. Offering to trade the weapon, while certainly illegal '
 'and indicative of intent to engage in criminal activity, does not meet the '
 'threshold of "use" as described in the statute. \n')

##############################################

Here is the completion with temperature completion_temp_1:
('ANSWER: No\n'
 '\n'
 'REASONING: While the statute covers "uses or carries," the specific facts '
 'only indicate that Mr. Smith offered to *trade* the weapon, not that he 

In [42]:
#Now, with the SCOTUS case summary, since it seemed to matter
answers = dict()
for temp in [0.5, 1, 1.5]:
    answers[f'completion_temp_{temp}'] = get_completion(prompt, temperature=temp)

In [43]:
for key, value in answers.items():
    print(f'Here is the completion with temperature {key}:')
    pprint(value)
    print('\n##############################################\n')

Here is the completion with temperature completion_temp_0.5:
('## ANSWER: Yes\n'
 '\n'
 '## REASONING: \n'
 '\n'
 "While Mr. Smith didn't fire or brandish the MAC-10 in a threatening manner, "
 'several factors suggest he "used" the firearm within the context of the '
 'statute:\n'
 '\n'
 '* **Active Employment:** He actively employed the MAC-10 as a bartering tool '
 'to facilitate a drug trafficking crime. He presented the firearm, discussed '
 'its capabilities, and offered it in exchange for cocaine. \n'
 '* **Integral to the Crime:** The possession and display of the MAC-10 were '
 "integral to the attempted drug transaction. He leveraged the firearm's value "
 'and desirability within the criminal context to obtain drugs.\n'
 '* **Dangerous Weapon:** The statute explicitly emphasizes the severity of '
 'using dangerous weapons like the MAC-10, a fully automatic firearm with high '
 'firing capacity, in connection with drug trafficking crimes. \n'
 '\n'
 "Therefore, a reasonable p